# Encoding one image with SGA

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ~/projects/improving-inference-for-neural-image-compression

/home/yiboyang/projects/improving-inference-for-neural-image-compression


In [3]:
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [4]:
import tensorflow as tf
import tensorflow_compression as tfc
import numpy as np
from absl import logging

import glob
import json

2023-03-15 23:34:34.890664: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
print(tf.version.VERSION)

2.10.0


In [6]:
def read_png(filename, channels=3):
    """Loads a PNG image file."""
    string = tf.io.read_file(filename)
    return tf.image.decode_image(string, channels=channels)

x = read_png("/home/yiboyang/data/kodak/kodim13.png")
x.shape, x.dtype

(TensorShape([512, 768, 3]), tf.uint8)

In [7]:
# Convert to float and add batch dim
x = tf.cast(x, tf.float32)
x = tf.expand_dims(x, 0)
x.shape

TensorShape([1, 512, 768, 3])

In [8]:
run_dir = 'tf2_checkpoints/coco/mbt2018-num_filters=256-lmbda=0.08/'
from mbt2018 import MBT2018Model as Model
import utils
args = utils.get_args_as_obj(glob.glob(os.path.join(run_dir, 'args-*.json'))[0])
model = Model.create_model(args)
ckpt = tf.train.latest_checkpoint(run_dir)
model.load_weights(ckpt)
# model.compile()
print('loaded weights from', ckpt)

loaded weights from tf2_checkpoints/coco/mbt2018-num_filters=256-lmbda=0.08/ckpt-lmbda=0.08-epoch=400-loss=2.580


In [9]:
res = model(x, training=False)
print(res['bpp'], res['psnr'])

tf.Tensor(2.4320767, shape=(), dtype=float32) tf.Tensor(36.090073, shape=(), dtype=float32)


In [10]:
%%time

from sga import default_sga_schedule
num_steps = 3000
(y, z), val_res = model.iterative_infer(x, num_steps, default_sga_schedule, learning_rate=1e-3, log_every_steps=100)

step=0, T=0.500 Train: loss=3.8182 mse=16.357 bpp=2.5097 psnr=35.9938	 Val: loss=3.7117 mse=15.996 bpp=2.4320 psnr=36.0907
step=100, T=0.500 Train: loss=3.7438 mse=15.637 bpp=2.4928 psnr=36.1893	 Val: loss=3.6741 mse=15.548 bpp=2.4303 psnr=36.2140
step=200, T=0.500 Train: loss=3.6766 mse=14.993 bpp=2.4772 psnr=36.3718	 Val: loss=3.6488 mse=15.231 bpp=2.4303 psnr=36.3035
step=300, T=0.500 Train: loss=3.6243 mse=14.506 bpp=2.4638 psnr=36.5154	 Val: loss=3.6322 mse=15.020 bpp=2.4306 psnr=36.3642
step=400, T=0.500 Train: loss=3.5899 mse=14.158 bpp=2.4573 psnr=36.6207	 Val: loss=3.6205 mse=14.868 bpp=2.4311 psnr=36.4083
step=500, T=0.500 Train: loss=3.5620 mse=13.877 bpp=2.4518 psnr=36.7077	 Val: loss=3.6110 mse=14.736 bpp=2.4321 psnr=36.4471
step=600, T=0.500 Train: loss=3.5415 mse=13.651 bpp=2.4494 psnr=36.7790	 Val: loss=3.6043 mse=14.645 bpp=2.4327 psnr=36.4738
step=700, T=0.500 Train: loss=3.5242 mse=13.487 bpp=2.4452 psnr=36.8315	 Val: loss=3.5977 mse=14.552 bpp=2.4335 psnr=36.5016
st